In [1]:
#Load libraries 
import pandas as pd
import sspa
import scipy
import numpy as np 

In [2]:
#Read in dataset
df = pd.read_csv("Data/Su_integrated_data.csv", index_col=0)

Check that data is centred at zero (mean=zero and standard deviation=1 for each analyte):

In [6]:
df_num  = df.iloc[:,:-2] #all rows, all columns apart from last two
print(df_num.max().max())
print(df_num.min().min())
print(df_num.mean(axis = 0)) #mean of 0
print(df_num.std(axis = 0)) #sd of 1

10.947848832735254
-7.68875111551009
1372     -6.938894e-18
16610    -4.163336e-17
72665     4.163336e-17
27823     0.000000e+00
30915    -4.163336e-17
              ...     
Q9Y4K4   -5.551115e-17
Q9Y5A7    4.163336e-17
Q9Y5L3   -6.938894e-18
Q9Y5V3   -3.469447e-17
Q9Y653    7.979728e-17
Length: 787, dtype: float64
1372      1.003929
16610     1.003929
72665     1.003929
27823     1.003929
30915     1.003929
            ...   
Q9Y4K4    1.003929
Q9Y5A7    1.003929
Q9Y5L3    1.003929
Q9Y5V3    1.003929
Q9Y653    1.003929
Length: 787, dtype: float64


In [7]:
df

,1372,16610,72665,27823,30915,37373,16831,545959,17050,16359,...,Q9ULX7,Q9UNK0,Q9Y478,Q9Y4K4,Q9Y5A7,Q9Y5L3,Q9Y5V3,Q9Y653,WHO_status,Group
sample_id,,,,,,,,,,,,,,,,,,,,,
INCOV001,-0.342522,-0.830841,-0.703451,-0.451278,0.153207,-0.443252,-0.228725,0.501998,-0.861886,-0.344155,...,0.056692,-1.339019,-1.482133,-1.836151,-1.708529,-0.677567,-0.982971,1.703613,3-4,COVID19
INCOV002,-0.125585,-1.002246,0.399273,-0.828341,-0.980576,0.373921,-1.076979,-0.131876,-0.869093,-0.467326,...,-0.589103,-0.916950,-0.999042,-0.323778,-0.368824,-1.212782,-0.263024,-0.830373,5-7,COVID19
INCOV003,-0.859326,-0.409159,3.573100,-0.546783,-0.993647,1.547649,0.370131,-0.020399,-0.703786,-0.015627,...,-1.357965,-1.209485,-1.103151,-0.608847,-0.821175,-1.010808,-0.662941,1.026187,5-7,COVID19
INCOV004,-0.562999,-1.560249,-1.269383,-1.437919,0.108734,-0.192066,0.161591,-0.586759,-1.457246,-0.500612,...,0.286096,-1.880875,-1.514350,-1.686500,-1.523233,-0.877769,-1.305748,0.806740,3-4,COVID19
INCOV005,-0.741957,0.387909,-0.711036,-0.711515,0.670720,0.610591,-0.322241,-0.793451,-1.230681,-0.246493,...,-0.092095,-1.600513,-1.340302,-1.155597,-1.447396,0.293662,-0.783788,-0.612823,3-4,COVID19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INCOV135,-2.075777,-0.485936,-0.671225,0.069483,0.042579,0.357250,-0.505272,-0.378441,-0.444915,-0.368108,...,1.412605,-0.208973,-0.638806,-0.789582,-0.693826,-0.374689,-1.150091,-1.031098,1-2,COVID19
INCOV136,-0.174602,1.167644,0.055499,0.113342,-0.663851,0.799367,0.004697,0.748894,1.328753,0.391221,...,0.796855,0.841765,1.359763,0.887098,1.239520,-0.831379,0.877162,1.989469,3-4,COVID19
INCOV137,0.274900,-0.014841,-0.928512,0.885263,-1.591862,0.230250,0.057818,-0.775962,-0.179700,-0.534786,...,0.518722,0.866393,0.878241,0.976843,1.191535,-0.619671,0.741753,-0.774813,5-7,COVID19


In [15]:
#Read in pathway file
mo_pathways = pd.read_csv("Data/Reactome_multi_omics_ChEBI_Uniprot.csv", index_col=0,dtype="str")
#Dtype warning because in some columns, some values are in string format whereas some are in integer format, that's why I specify dtype="str"

In [16]:
#Download the root pathways
root_path = pd.read_excel('Data/Root_pathways.xlsx', header=None)
root_pathway_dict = {root_path[0][i]:root_path[1][i] for i in range(0,len(root_path))}
root_pathway_names = list(root_pathway_dict.keys())

In [17]:
#Test kPCA on whole dataset
kpca_scores_all = sspa.sspa_kpca(df, mo_pathways) #737 pathways

#Remove root pathways first
kpca_scores_all = kpca_scores_all.drop(columns = list(set(root_pathway_names) & set(kpca_scores_all.columns))) #using Sara's code to drop root pathways

kpca_scores_all #710 pathways

,R-HSA-109581,R-HSA-109606,R-HSA-109704,R-HSA-110331,R-HSA-110362,R-HSA-110373,R-HSA-111465,R-HSA-112310,R-HSA-112311,R-HSA-112314,...,R-HSA-9818027,R-HSA-9824439,R-HSA-9824443,R-HSA-9824446,R-HSA-983168,R-HSA-983169,R-HSA-983231,R-HSA-983695,R-HSA-983705,R-HSA-983712
sample_id,,,,,,,,,,,,,,,,,,,,,
INCOV001,0.413536,0.729067,0.303770,-0.376183,-0.430659,-0.430659,0.384273,0.023789,0.577936,0.597223,...,0.644510,-0.481606,-0.150567,0.372096,-0.133388,0.285130,-0.434451,0.645959,0.368779,0.658221
INCOV002,0.362645,0.055357,0.527061,-0.311560,0.638987,0.638987,0.444671,-0.092256,0.604496,0.701297,...,0.491841,-0.548186,-0.533311,0.341016,-0.303490,0.433799,-0.539958,0.410963,0.457822,0.432791
INCOV003,0.575862,0.575287,0.459731,-0.308065,-0.082800,-0.082800,0.736859,0.016257,0.304684,0.712081,...,0.653922,-0.433432,-0.351057,0.286416,-0.620820,0.479704,-0.654156,0.460245,0.419668,0.335348
INCOV004,0.527632,0.486186,0.153415,0.664277,-0.377176,-0.377176,0.479131,0.165562,0.615549,0.232496,...,0.482319,-0.165577,-0.402658,0.341416,-0.233868,0.343820,-0.372565,0.439250,0.565834,0.339853
INCOV005,0.642278,0.738515,0.411455,0.265643,-0.157652,-0.157652,0.727463,0.400127,0.536780,0.648969,...,0.647857,-0.471051,-0.522629,0.527788,-0.417845,0.582129,-0.647128,0.708149,0.685953,0.646138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INCOV135,0.232999,0.578294,0.073997,-0.394520,-0.571496,-0.571496,0.571166,0.259495,0.466705,0.383918,...,0.321763,-0.344207,0.141510,0.326663,-0.452698,0.433485,-0.559021,0.518424,0.529108,0.463221
INCOV136,-0.381783,-0.526387,-0.379957,-0.206822,0.048728,0.048728,-0.514031,0.033658,-0.447514,-0.426311,...,-0.458273,0.585416,0.434440,-0.353734,0.336194,-0.207547,0.679397,-0.546947,-0.438766,-0.607344
INCOV137,-0.397495,-0.414259,-0.125324,-0.342753,-0.217334,-0.217334,-0.433742,0.160082,-0.328729,-0.576714,...,-0.543902,0.645635,0.115486,-0.211909,0.414120,-0.248971,0.696940,-0.391584,-0.403203,0.114467


In [18]:
#Separate dataset into two groups
df_mild = (df[df["WHO_status"] == '1-2']).iloc[:,:-2] #45 samples, remove the metadata
df_severe = (df[(df["WHO_status"] == '3-4') | (df["WHO_status"] == '5-7')]).iloc[:,:-2] #83 samples

### Step 1: Determine initial test-statistic

In [20]:
#Function to calculate the squared Spearman correlation matrix 

def squared_spearman_corr(data):
    kpca_scores = sspa.sspa_kpca(data, mo_pathways)   
    kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns))) #using Sara's code to drop root pathways

    spearman_results = scipy.stats.spearmanr(kpca_scores)
    squared_spearman_coef = np.square(spearman_results[0]) #correlation coefficients (spearman_results[1] gives the p-values)

    return squared_spearman_coef,list(kpca_scores.columns)




#Function to calculate the absolute difference between two matrices and then determine the mean for each edge

def absolute_val(data1,data2,edgelist):
    abs_rho_squared = np.absolute(np.array(data1) - np.array(data2))

    #Mask the upper half of the dataframe (so I don't view the comparisons between the two same genes, and also the duplicate comparisons are removed)
    mask = abs_rho_squared.copy()
    mask = np.triu(np.ones(mask.shape)).astype(bool)
    mask = np.invert(mask) #invert true and false values so the diagonal is False as well
    non_dup_abs_squared = pd.DataFrame(abs_rho_squared, columns = edgelist, index = edgelist)
    non_dup_abs_squared = pd.DataFrame(non_dup_abs_squared).where(mask) #Replace all false values with NaN using mask

    abs_squared_list = non_dup_abs_squared.stack().reset_index()
    abs_squared_list['level_0'] = abs_squared_list["level_0"].astype(str) + ", " + abs_squared_list['level_1']
    abs_squared_list.columns = ["Edges","na","Initial_tstat"]
    abs_squared_list.index = abs_squared_list["Edges"]
    abs_squared_list = abs_squared_list.drop(columns = ["Edges","na"])

    return(abs_squared_list)

Note: For the delta squared correlation values for the unshuffled data (i.e. the real data) I keep the indices (pathway edges). Since I already have a record of the edges, there is no need to keep the edges for each permutation, since the order is the same each time. 

In [21]:
spearman_mild,edgelist = squared_spearman_corr(df_mild)
spearman_severe,edgelist = squared_spearman_corr(df_severe)

output = absolute_val(spearman_mild,spearman_severe,edgelist)

In [24]:
output  #251695 pathway pairs (because of 710 pathways)

,Initial_tstat
Edges,
"R-HSA-109606, R-HSA-109581",0.214095
"R-HSA-109704, R-HSA-109581",0.442899
"R-HSA-109704, R-HSA-109606",0.308395
"R-HSA-110331, R-HSA-109581",0.112487
"R-HSA-110331, R-HSA-109606",0.133920
"R-HSA-110331, R-HSA-109704",0.282229
"R-HSA-110362, R-HSA-109581",0.091419
"R-HSA-110362, R-HSA-109606",0.028304
"R-HSA-110362, R-HSA-109704",0.053538


In [25]:
#output.to_csv("Data/permutation_test_files_integrated/initial_tstats.csv")

### Step 2: Shuffle the labels

### Step 3: Read in the permutation files 

### Step 4: Compare the difference in edges with other networks

Comparing with the significant edges in the proteomic differential network (after FDR correction) and the metabolomic differential network (before FDR correction because only 10 edges afterwards):

In [3]:
def edge_num (omics):
    with open('Data/permutation_test_files_'+omics+'/sig_edges.txt') as f:
    #with open('Data/permutation_test_files_'+omics+'/sig_edges_beforefdr.txt') as f:    #for metabolomic data since there are so few edges left
        lines = f.readlines()
    edges_remaining = []
    edges = lines[0].split(",")

    for index in range(0,len(edges),2):
        list1 = edges[index],(edges[index+1][1:]) #becomes a tuple
        edges_remaining.append(list1)

    return edges_remaining


In [4]:
proteomic_edges = edge_num('proteomics')
integrated_edges = edge_num('integrated')
metabolomic_edges = edge_num('metabolomics')

In [5]:
len(integrated_edges)

26761

In [6]:
intersection = list(set(proteomic_edges).intersection(list(set(integrated_edges))))  
len(intersection) 

22728

Comparing the initial test statistics of the integrated data with the proteomic initial test statistics:

In [7]:
#Load dataset
metabolomic = pd.read_csv('Data/permutation_test_files_metabolomics/initial_tstats.csv', index_col=0)
proteomic = pd.read_csv('Data/permutation_test_files_proteomics/initial_tstats.csv', index_col=0)
integrated = pd.read_csv('Data/permutation_test_files_integrated/initial_tstats.csv', index_col=0)

In [8]:
metabolomic = metabolomic.rename(columns={'Initial_tstat': 'metabolomic_tstat'})
proteomic = proteomic.rename(columns={'Initial_tstat': 'proteomic_tstat'})
integrated = integrated.rename(columns={'Initial_tstat': 'integrated_tstat'})

In [77]:
integrated

,integrated_tstat
Edges,
"R-HSA-109606, R-HSA-109581",0.214095
"R-HSA-109704, R-HSA-109581",0.442899
"R-HSA-109704, R-HSA-109606",0.308395
"R-HSA-110331, R-HSA-109581",0.112487
"R-HSA-110331, R-HSA-109606",0.133920
...,...
"R-HSA-983712, R-HSA-983168",0.407463
"R-HSA-983712, R-HSA-983169",0.499627
"R-HSA-983712, R-HSA-983231",0.492987


In [9]:
initial_tstat_comp = proteomic.join(integrated)
initial_tstat_comp

,proteomic_tstat,integrated_tstat
Edges,,
"R-HSA-109606, R-HSA-109581",0.214095,0.214095
"R-HSA-109704, R-HSA-109581",0.442899,0.442899
"R-HSA-109704, R-HSA-109606",0.308395,0.308395
"R-HSA-110362, R-HSA-109581",0.091419,0.091419
"R-HSA-110362, R-HSA-109606",0.028304,0.028304
...,...,...
"R-HSA-983705, R-HSA-9824446",0.046766,0.068394
"R-HSA-983705, R-HSA-983168",0.029076,0.029076
"R-HSA-983705, R-HSA-983169",0.065720,0.065720


In [10]:
#Number of pathway pairs with initial test statistic that are the same

metabolomic_influence_edge = []
counter = 0

for edge in initial_tstat_comp.index:
    if initial_tstat_comp.loc[edge].proteomic_tstat == initial_tstat_comp.loc[edge].integrated_tstat:
        counter += 1
    else:
        metabolomic_influence_edge.append(edge)


print(counter)

#166,753 edges of the proteomic initial test statistic dataset are all included in the integrated initial test statistic data
#117,381 edges out of 166,753 are identical
#Therefore 49,372 edges were influenced by the metabolomic data

len(metabolomic_influence_edge)

117381


49372

In [11]:
metabolomic_influence_edge_tuple = []
for i in range(len(metabolomic_influence_edge)):
    edge = metabolomic_influence_edge[i].split(",")
    edge = (edge[0],edge[1][1:])
    metabolomic_influence_edge_tuple.append(edge)

In [161]:
metabolomic_influence_edge_tuple

[('R-HSA-112315', 'R-HSA-109581'),
 ('R-HSA-112315', 'R-HSA-109606'),
 ('R-HSA-112315', 'R-HSA-109704'),
 ('R-HSA-112315', 'R-HSA-110362'),
 ('R-HSA-112315', 'R-HSA-110373'),
 ('R-HSA-112315', 'R-HSA-111465'),
 ('R-HSA-112315', 'R-HSA-112314'),
 ('R-HSA-112399', 'R-HSA-112315'),
 ('R-HSA-114508', 'R-HSA-109581'),
 ('R-HSA-114508', 'R-HSA-109606'),
 ('R-HSA-114508', 'R-HSA-109704'),
 ('R-HSA-114508', 'R-HSA-110362'),
 ('R-HSA-114508', 'R-HSA-110373'),
 ('R-HSA-114508', 'R-HSA-111465'),
 ('R-HSA-114508', 'R-HSA-112314'),
 ('R-HSA-114508', 'R-HSA-112315'),
 ('R-HSA-114508', 'R-HSA-112399'),
 ('R-HSA-114608', 'R-HSA-112315'),
 ('R-HSA-114608', 'R-HSA-114508'),
 ('R-HSA-1168372', 'R-HSA-112315'),
 ('R-HSA-1168372', 'R-HSA-114508'),
 ('R-HSA-1169091', 'R-HSA-112315'),
 ('R-HSA-1169091', 'R-HSA-114508'),
 ('R-HSA-1169408', 'R-HSA-112315'),
 ('R-HSA-1169408', 'R-HSA-114508'),
 ('R-HSA-1169410', 'R-HSA-112315'),
 ('R-HSA-1169410', 'R-HSA-114508'),
 ('R-HSA-1222556', 'R-HSA-112315'),
 ('R-HSA-12

In [109]:
list(proteomic_edges)

[('R-HSA-112314', 'R-HSA-109581'),
 ('R-HSA-112315', 'R-HSA-109581'),
 ('R-HSA-114508', 'R-HSA-109581'),
 ('R-HSA-1168372', 'R-HSA-111465'),
 ('R-HSA-1168372', 'R-HSA-112314'),
 ('R-HSA-1168372', 'R-HSA-112315'),
 ('R-HSA-1168372', 'R-HSA-114508'),
 ('R-HSA-1169091', 'R-HSA-109606'),
 ('R-HSA-1169091', 'R-HSA-111465'),
 ('R-HSA-1169091', 'R-HSA-112314'),
 ('R-HSA-1169091', 'R-HSA-112315'),
 ('R-HSA-1169091', 'R-HSA-114508'),
 ('R-HSA-1169408', 'R-HSA-109581'),
 ('R-HSA-1169408', 'R-HSA-109606'),
 ('R-HSA-1169408', 'R-HSA-111465'),
 ('R-HSA-1169408', 'R-HSA-1168372'),
 ('R-HSA-1169408', 'R-HSA-1169091'),
 ('R-HSA-1169410', 'R-HSA-109581'),
 ('R-HSA-1169410', 'R-HSA-109606'),
 ('R-HSA-1169410', 'R-HSA-111465'),
 ('R-HSA-1169410', 'R-HSA-1168372'),
 ('R-HSA-1169410', 'R-HSA-1169091'),
 ('R-HSA-1226099', 'R-HSA-109581'),
 ('R-HSA-1226099', 'R-HSA-1168372'),
 ('R-HSA-1226099', 'R-HSA-1169091'),
 ('R-HSA-1227986', 'R-HSA-112314'),
 ('R-HSA-1227986', 'R-HSA-112315'),
 ('R-HSA-1227986', 'R-HSA

In [17]:
#Need to see proportion of metabolomic-influenced edges that are significant edges in the proteomic data vs the integrated data
print(len(metabolomic_influence_edge))
print(len(list(set(proteomic_edges).intersection(set(metabolomic_influence_edge_tuple))))) #9616 edges significant
print(len(list(set(integrated_edges).intersection(set(metabolomic_influence_edge_tuple))))) #After integrating with the metabolomic data, 8079 significant
print(len(list(set(integrated_edges).intersection(set(proteomic_edges)))))

49372
9616
8079
22728


In [18]:
initial_tstat_comp = metabolomic.join(integrated)
initial_tstat_comp

,metabolomic_tstat,integrated_tstat
Edges,,
"R-HSA-112310, R-HSA-110331",0.007044,0.007044
"R-HSA-112311, R-HSA-110331",0.023605,0.079169
"R-HSA-112311, R-HSA-112310",0.285484,0.026959
"R-HSA-112315, R-HSA-110331",0.008600,0.000009
"R-HSA-112315, R-HSA-112310",0.057525,0.812476
...,...,...
"R-HSA-975634, R-HSA-9717207",0.035746,0.051379
"R-HSA-975634, R-HSA-9734207",0.016982,0.042406
"R-HSA-975634, R-HSA-9735804",0.000969,0.077219


In [19]:
#Number of pathway pairs with initial test statistic that are the same

proteomic_influence_edge = []
counter = 0
for edge in initial_tstat_comp.index:
    if initial_tstat_comp.loc[edge].metabolomic_tstat == initial_tstat_comp.loc[edge].integrated_tstat:
        counter += 1
    else:
        proteomic_influence_edge.append(edge)

print(counter)

#10,296 edges of the metabolomic initial test statistic dataset are all included in the integrated initial test statistic data
#2,088 edges out of 10,296 are identical
#Therefore 8,208 edges were influenced by the proteomic data

proteomic_influence_edge

2088


['R-HSA-112311, R-HSA-110331',
 'R-HSA-112311, R-HSA-112310',
 'R-HSA-112315, R-HSA-110331',
 'R-HSA-112315, R-HSA-112310',
 'R-HSA-112315, R-HSA-112311',
 'R-HSA-1237112, R-HSA-112311',
 'R-HSA-1237112, R-HSA-112315',
 'R-HSA-1368082, R-HSA-112311',
 'R-HSA-1368082, R-HSA-112315',
 'R-HSA-1368108, R-HSA-112311',
 'R-HSA-1368108, R-HSA-112315',
 'R-HSA-1428517, R-HSA-110331',
 'R-HSA-1428517, R-HSA-112310',
 'R-HSA-1428517, R-HSA-112311',
 'R-HSA-1428517, R-HSA-112315',
 'R-HSA-1428517, R-HSA-1237112',
 'R-HSA-1428517, R-HSA-1368082',
 'R-HSA-1428517, R-HSA-1368108',
 'R-HSA-1483148, R-HSA-112311',
 'R-HSA-1483148, R-HSA-112315',
 'R-HSA-1483148, R-HSA-1428517',
 'R-HSA-1483206, R-HSA-110331',
 'R-HSA-1483206, R-HSA-112310',
 'R-HSA-1483206, R-HSA-112311',
 'R-HSA-1483206, R-HSA-112315',
 'R-HSA-1483206, R-HSA-1237112',
 'R-HSA-1483206, R-HSA-1368082',
 'R-HSA-1483206, R-HSA-1368108',
 'R-HSA-1483206, R-HSA-1428517',
 'R-HSA-1483206, R-HSA-1483148',
 'R-HSA-1483255, R-HSA-110331',
 'R-

In [20]:
proteomic_influence_edge_tuple = []
for i in range(len(proteomic_influence_edge)):
    edge = metabolomic_influence_edge[i].split(",")
    edge = (edge[0],edge[1][1:])
    proteomic_influence_edge_tuple.append(edge)

In [21]:
#Need to see proportion of proteomic-influenced edges that are significant edges in the metabolomic data vs the integrated data
print(len(proteomic_influence_edge))
print(len(list(set(metabolomic_edges).intersection(set(proteomic_influence_edge_tuple))))) #0 edges significant
print(len(list(set(integrated_edges).intersection(set(proteomic_influence_edge_tuple))))) #After integrating with the proteomic data, 1272 significant
print(len(list(set(integrated_edges).intersection(set(metabolomic_edges)))))

8208
0
1272
2


In [39]:
metabolomic_df = pd.read_csv('Data/Su_COVID_metabolomics_processed_commoncases.csv', index_col=0)
proteomic_df = pd.read_csv('Data/Su_COVID_proteomics_processed_commoncases.csv', index_col=0)
integrated_df = pd.read_csv("Data/Su_integrated_data.csv", index_col=0)


#Remove root pathways
root_path = pd.read_excel('Data/Root_pathways.xlsx', header=None)
root_pathway_names = list(root_pathway_dict.keys())

In [42]:
reactome_pathways = sspa.process_gmt("Data/Reactome_Homo_sapiens_pathways_compounds_R84.gmt")
kpca_scores = sspa.sspa_kpca(metabolomic_df.iloc[:,:-2], reactome_pathways)

#Using Sara's code
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))

metabolomic_pathways = kpca_scores.columns
metabolomic_pathways

Index(['R-HSA-110331', 'R-HSA-112310', 'R-HSA-112311', 'R-HSA-112315',
       'R-HSA-1237112', 'R-HSA-1368082', 'R-HSA-1368108', 'R-HSA-1428517',
       'R-HSA-1483148', 'R-HSA-1483206',
       ...
       'R-HSA-9707564', 'R-HSA-9707616', 'R-HSA-9711123', 'R-HSA-9717189',
       'R-HSA-9717207', 'R-HSA-9734207', 'R-HSA-9735804', 'R-HSA-9749641',
       'R-HSA-9753281', 'R-HSA-975634'],
      dtype='object', length=144)

In [36]:
#Load in a file downloaded from https://reactome.org/download/current/UniProt2Reactome_All_Levels.txt
reactome_pathways = sspa.process_reactome('Homo sapiens', infile = 'Data/UniProt2Reactome_All_Levels.txt', download_latest = False, filepath = None)
kpca_scores = sspa.sspa_kpca(proteomic_df.iloc[:,:-2], reactome_pathways)


#Using Sara's code
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))

proteomic_pathways = kpca_scores.columns
proteomic_pathways

Index(['R-HSA-109581', 'R-HSA-109606', 'R-HSA-109704', 'R-HSA-110362',
       'R-HSA-110373', 'R-HSA-111465', 'R-HSA-112314', 'R-HSA-112315',
       'R-HSA-112399', 'R-HSA-114508',
       ...
       'R-HSA-9772573', 'R-HSA-9818027', 'R-HSA-9824439', 'R-HSA-9824443',
       'R-HSA-9824446', 'R-HSA-983168', 'R-HSA-983169', 'R-HSA-983231',
       'R-HSA-983695', 'R-HSA-983705'],
      dtype='object', length=578)

In [37]:
reactome_pathways = pd.read_csv("Data/Reactome_multi_omics_ChEBI_Uniprot.csv", index_col=0,dtype="str")

kpca_scores = sspa.sspa_kpca(integrated_df.iloc[:,:-2], reactome_pathways)

#Remove root pathways
#Convert pathway ID to name
root_path = pd.read_excel('Data/Root_pathways.xlsx', header=None)
root_pathway_dict = {root_path[0][i]:root_path[1][i] for i in range(0,len(root_path))}

root_pathway_names = list(root_pathway_dict.keys())
#Using Sara's code
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))

integrated_pathways = kpca_scores.columns
integrated_pathways

Index(['R-HSA-109581', 'R-HSA-109606', 'R-HSA-109704', 'R-HSA-110331',
       'R-HSA-110362', 'R-HSA-110373', 'R-HSA-111465', 'R-HSA-112310',
       'R-HSA-112311', 'R-HSA-112314',
       ...
       'R-HSA-9818027', 'R-HSA-9824439', 'R-HSA-9824443', 'R-HSA-9824446',
       'R-HSA-983168', 'R-HSA-983169', 'R-HSA-983231', 'R-HSA-983695',
       'R-HSA-983705', 'R-HSA-983712'],
      dtype='object', length=710)

In [48]:
intersection = list(set(metabolomic_pathways).intersection(list(set(proteomic_pathways))))   #47 pathways
intersection = (set(metabolomic_pathways) & set(proteomic_pathways) & set(integrated_pathways))  #47 pathways 

intersection = list(set(metabolomic_pathways).intersection(list(set(integrated_pathways))))  #144 pathways (same as number of metabolomic)
intersection = list(set(proteomic_pathways).intersection(list(set(integrated_pathways))))  #578 pathways (same as number of proteomic)

#710 pathways in integrated dataset
#((144+578)-47+35)

len(list(set(integrated_pathways) - (set(metabolomic_pathways)|set(proteomic_pathways)))) #35 pathways for integrated data not detected by either single omics